In [32]:
import wrds
import csv
from datetime import datetime, timedelta, date
import sys
import statsmodels.formula.api as sm
import pandas as pd
from google.colab import drive 
import holidays
import pandas_market_calendars as mcal
drive.mount("/content/gdrive")

#To max out field limit
csv.field_size_limit(sys.maxsize)

db.close()

db = wrds.Connection(wrds_username='holzeral')

# Function to check whether date is on a non-trading day. If so, then return next trading day.

holidaysUS = holidays.US()
nyse = mcal.get_calendar('NYSE')
stock_holidays = nyse.holidays()

stock_holidays = list(pd.to_datetime(stock_holidays.holidays))
stock_holidays = [x.date() for x in stock_holidays]

tickers = ['AAPL', 'AMZN', 'TSLA', 'FB', 'BA', 'NFLX', 'DIS', 'EFX', 'BAC', 'INTC', 'F', 'GE', 'GM', 
 'MSFT', 'SBUX', 'AIR', 'AAL', 'IBM', 'JPM', 'CMG', 'WFC', 'C', 'TWTR', 'WMT', 'MCD', 'AMD', 'NVDA', 'JNJ', 'GS', 
 'BABA', 'CAT', 'MU', 'CSCO', 'XOM', 'CVX', 'BP', 'GOOGL', 'GPRO', 'COST', 'HD', 'NKE', 'KO', 
 'AXP', 'TGT', 'ATVI', 'CMCSA', 'DAL', 'LMT', 'T', 'ABBV', 'PFE', 'GILD', 'ADBE', 'CRM', 'VZ', 'AVGO', 'BX', 
 'LULU', 'BLK', 'UNH', 'KMI', 'BBY', 'PG', 'AGI', 'AMAT', 'MRK', 'M', 'BIDU', 'QCOM', 'FDX', 'AMGN', 
 'BMY', 'ORCL', 'BHP', 'MA', 'KR', 'MO', 'GME', 'PM', 'CHK', 'MMM', 'BBBY', 'COP', 'IRBT', 'MS', 
 'FCX', 'HAL', 'AGN', 'HPQ', 'UAL', 'CELG', 'JWN', 'CVS', 'V', 'EA', 'STZ', 'GLW', 'ADP', 'APC', 'AZN']

def check_trading_dayandhour(day):
    trading_day = day
    while trading_day.weekday() in holidays.WEEKEND or trading_day in stock_holidays:
        trading_day += timedelta(1)
    return trading_day

#Output path
output_file_path = "gdrive/My Drive/Thesis/processed data/"
output_file_name = "data_whole_year5.csv"

#Get all dates between 2015 and 2019

start_date = date(2015, 1, 1)
end_date = date(2019, 12, 31)
delta = end_date - start_date
dates = []

for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    dates.append(day)

#List of 100 most frequent stock ticker symbols in NASDAQ news data set between 2015 and 2019

data_fill = []
i = 0

for tick in tickers[99:]:

    for date in dates:

        print(tick)

        date_adj = check_trading_dayandhour(date)

        ########## Following code uses wrds database to extract data for control variable: share turnover and nasdaq dummy ##########

        # Substract days from datetime variable to get start and end date in API request
        day0 = date_adj - timedelta(0)
        dayminus1 = date_adj - timedelta(1)
        dayminus252 = date_adj - timedelta(252)
        dayminus6 = date_adj - timedelta(6)

        # Convert to proper time format for wrds request
        day0 = day0.strftime("%m/%d/%Y")
        dayminus1 = dayminus1.strftime("%m/%d/%Y")
        dayminus6 = dayminus6.strftime("%m/%d/%Y")
        dayminus252 = dayminus252.strftime("%m/%d/%Y")

        permno = db.raw_sql("""select permno, date
                                                        from crsp.dse
                                                        where TICKER = '{}'
                                                        and date  <= '{}'""".format(tick, day0))
        

        #Get latest permno
        permno["date"] = pd.to_datetime(permno["date"], format = "%Y/%m/%d")
        permno = permno.sort_values(by = "date")
        permno = permno["permno"].iloc[-1]

        # data query to get control variable share turnover: volume within (-252,-6) divided by shares outstanding on 0
        voldata = db.raw_sql("""select vol 
                                                    from crsp.dsf
                                                    where permno = {}
                                                    and date between '{}' and '{}'""".format(permno, dayminus252,
                                                                                             dayminus6))

        shrout_m1 = db.raw_sql("""select shrout
                                                    from crsp.dsf
                                                    where permno = {}
                                                    and date between '{}' and '{}'""".format(permno, dayminus6, day0))
        
        issuno = db.raw_sql("""select issuno
                                                    from crsp.dsf
                                                    where permno = {}
                                                    and date between '{}' and '{}'""".format(permno, dayminus252, day0))    
                                            
        if int(issuno["issuno"].iloc[-1]) != 0:
            issuno_i = 1
        else:
            issuno_i = 0

        print(shrout_m1)

        shareturnover = (int(voldata["vol"].sum()) / (int(shrout_m1["shrout"][0]) * 1000))

        ########## Following code uses wrds database to extract data for control variable: size  ##########

        # Size defined as price * shrout on date -1
        price_and_shroutm1 = db.raw_sql("""select prc, shrout
                                                            from crsp.dsf
                                                            where permno = {}
                                                            and date between '{}' and '{}'""".format(permno, dayminus1,day0))
        

        size = price_and_shroutm1["prc"][0] * int(price_and_shroutm1["shrout"][0])


        ########## Following code uses wrds database to extract data for control variable: BTM  ##########

        dayplus5years = date + timedelta(1825)
        dayplus5years = dayplus5years.strftime("%m/%d/%Y")

        assets_and_equity = db.raw_sql("""select tic, datadate, seq
                                                            from comp.funda
                                                            where tic = '{}'
                                                            and datadate between '{}' and '{}'""".format(tick,
                                                                                                         day0,
                                                                                                         dayplus5years))
        assets_and_equity = assets_and_equity.dropna()

        # BTM defined as book value of total stockholder equity divided by MVE

        btm = (assets_and_equity["seq"].iloc[0] / (size / 1000))

        ########## Following code uses wrds database to extract data for control variable: pref_alpha  ##########

        ret = db.raw_sql("""select date, ret 
                                                                        from crsp.dsf
                                                                        where permno = {}
                                                                        and date between '{}' and '{}'""".format(permno,
                                                                                                                 dayminus252,
                                                                                                                 dayminus6))

        ff = db.raw_sql("""select date, mktrf, smb, hml, rf
                                                                        from ff.factors_daily
                                                                        where date between '{}' and '{}'""".format(
            dayminus252,
            dayminus6))

        # Change fetched data to pandas dataframe
        ff = pd.DataFrame(ff, columns=["date", "mktrf", "smb", "hml", "rf"])
        ret = pd.DataFrame(ret, columns=["date", "ret"])

        # Merge the two dataframes on date and drop NAs so only dates with no missing values are included in the regression
        data_reg = ret.merge(ff,
                             on="date",
                             how="left")

        data_reg = data_reg[data_reg.notna()]

        # Substract risk free rate from daily return to get excess return
        data_reg["excess_return"] = data_reg["ret"] - data_reg["rf"]
        data_reg = data_reg.drop(["rf", "date"], axis=1)

        if (len(data_reg)) >= 60:
          model = sm.ols("excess_return ~ mktrf + smb + hml", data=data_reg).fit()
          pre_falpha = model.params["Intercept"]
        else:
          raise ValueError("Data incomplete")

        ########## Following code uses wrds database to extract data for dependent variable: excess returns day period: 0 to 3  ##########
        # Excess return defined as 2 day period buy and hold return from day 0 to day 1 minus cumul value weighted CRSP return in this period

        # Last day of buy and hold period. If a Friday, then consider Friday and Monday as two day event period
        if date_adj.weekday() != 4:
            dayplus1 = date_adj + timedelta(1)
        else:
            dayplus1 = date_adj + timedelta(3)

        dayplus1 = check_trading_dayandhour(dayplus1)
        dayplus1 = dayplus1.strftime("%m/%d/%Y")

        # Fetch prices and returns for given buy and hold period
        exc_ret = db.raw_sql("""select date, prc, ret
                                                            from crsp.dsf
                                                            where permno = {}
                                                            and date between '{}' and '{}'""".format(permno, day0,
                                                                                                     dayplus1))

        vw_ret = db.raw_sql("""select date, vwretd
                                                            from crsp.dsi
                                                            where date between '{}' and '{}'""".format(day0, dayplus1))
        
        print(exc_ret)

        # Initialize cumulative return variable to 1, as we need to multiply returns of each given day in the buy and hold period
        cumul_return = 1
        vw_cumul_return = 1
        for j in range(len(exc_ret["ret"])):
                cumul_return *= (1 + exc_ret["ret"][j])
                vw_cumul_return *= (1 + vw_ret["vwretd"][j])

        print(cumul_return)

        cum_ab_return = cumul_return - vw_cumul_return

        data_fill.append([date, tick, issuno_i, shareturnover,
                      size, btm, pre_falpha,
                      cum_ab_return])
        
        i += 1
        print(i)

df = pd.DataFrame(data_fill,
                  columns=["Date", "Ticker", "Nasdaq",
                           "Turnover",
                           "Size", "BTM", "pref_alpha", "CAR"])

with open(output_file_path + output_file_name, "w") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(
        ["Date", "Ticker", "Nasdaq", "Turnover", "Size", "BTM",
         "pref_alpha", "CAR"])
    for index, row in df.iterrows():
        writer.writerow(row)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Loading library list...
Done
100
AZN
     shrout
0  168028.0
1  168028.0
2  120623.0
3  120623.0
         date        prc       ret
0  2015-01-02  70.400002  0.000284
1  2015-01-05  69.860001 -0.007670
0.9926115848960442
1
AZN
     shrout
0  168028.0
1  168028.0
2  120623.0
3  120623.0
         date        prc       ret
0  2015-01-02  70.400002  0.000284
1  2015-01-05  69.860001 -0.007670
0.9926115848960442
2
AZN
     shrout
0  168028.0
1  120623.0
2  120623.0
3  120623.0
         date        prc       ret
0  2015-01-05  69.860001 -0.007670
1  2015-01-06  68.160004 -0.024334
0.9681818490749593
3
AZN
     shrout
0  168028.0
1  120623.0
2  120623.0
3  120623.0
         date        prc       ret
0  2015-01-05  69.860001 -0.007670
1  2015-01-06  68.160004 -0.024334
0.9681818490749593
4
AZN
     shrout
0  168028.0
1  120623.0
2  120623.0
3  120623.0
         dat

KeyboardInterrupt: ignored

In [2]:
!pip install wrds
!pip install holidays
!pip install pandas_market_calendars

     |████████████████████████████████| 3.0 MB 8.2 MB/s 
     |████████████████████████████████| 89 kB 3.3 MB/s 
     |████████████████████████████████| 150 kB 10.7 MB/s 
  Created wheel for exchange-calendars: filename=exchange_calendars-3.6.1-py3-none-any.whl size=180505 sha256=cd659d19443c4aeca86388d0f1066eb9c1131fb3f526c9f7a95d1a48265d3502
  Stored in directory: /root/.cache/pip/wheels/db/70/a2/5e1d8d0873feb8cb9808f3b55e8f270698b742db54a4b8ad2e
Successfully built exchange-calendars
